In [83]:
from numpy import genfromtxt
import librosa
from datetime import datetime
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [9]:


begin_time = datetime.now()
i=1
workdir = os.path.join(os.getcwd(), 'songs')
filename = os.path.join(workdir, 'song_{}.mp3'.format(i))
print(filename)
x, sr = librosa.load(filename)
chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=100, n_chroma=50)

print(datetime.now() - begin_time)

#my_data = genfromtxt(os.path.join(features_wd, 'full_chroma_1.csv'), delimiter=',')

/Users/arunravishankar/ragam_identification/ragam_identification/songs/song_1.mp3


/Users/arunravishankar/opt/anaconda3/envs/ragam_id/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


0:00:11.058435


In [14]:
begin_time = datetime.now()
mfccs_1, chroma_1, mel_1, contrast_1, tonnetz_1 = extract_feature(filename)
print(datetime.now()-begin_time)

/Users/arunravishankar/opt/anaconda3/envs/ragam_id/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


0:00:21.031243


In [21]:
df = pd.read_csv('sample_50_rand_df.csv')

In [24]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Concert ID,Track,Kriti,Ragam,Composer,Main Artist,Album hrefs,Download URLs
0,1828,3369,36439,01 - C4133 - 4079-T_Sashidar-Flut,2,entani_nE_varNintunu,mukhAri,tyAgarAja,T Sashidar,https://www.sangeethamshare.org/tvg/UPLOADS-40...,https://www.sangeethamshare.org/fstream.php?fi...
1,2425,4444,42092,36 - C0001 - DKJ-MYSORE,13,ranjani_mrdu_pankaja,rAgamAlikA,tanjAvUr_sankara_iyer,DK Jayaraman,https://www.sangeethamshare.org/ramakrishnan_c...,https://www.sangeethamshare.org/fstream.php?fi...
2,3524,6502,9553,09 - C0138 - SSI--048-1974,5,ADamODi_galadE_rAmayya,cArukEsi,tyAgarAja,Semmangudi Srinivasa Iyer,https://www.sangeethamshare.org/ksj/Semmangudi...,https://www.sangeethamshare.org/fstream.php?fi...
3,5381,9804,14793,03 - C0393 - 252-TN_shEshagOpAlan,8,mAnamu_lEdA,hamIrkalyANi,tyAgarAja,TN Seshagopalan,https://www.sangeethamshare.org/manjunath/Carn...,https://www.sangeethamshare.org/fstream.php?fi...
4,4911,9048,9942,03 - C0801 - 449-TN_kriShNan-Dr_N,7,sakhi_prANa_sakhuDiTu,cenjuruTi,dharmapuri_subbarAya_iyer,TN Krishnan,https://www.sangeethamshare.org/manjunath/Carn...,https://www.sangeethamshare.org/fstream.php?fi...


In [27]:
from sklearn.model_selection import train_test_split
X_train_file, X_test_file, y_train_file, y_test_file = train_test_split(df.index, 
                                                    df['Ragam'], 
                                                    test_size = 0.1, 
                                                    random_state = 0,
                                                    stratify = df['Ragam'])
indices = list(X_train_file)
ragams = list(y_train_file)
print(indices[0], ragams[0], df.loc[indices[0]]['Ragam'])

5042 shankarAbharaNam shankarAbharaNam


In [143]:
indices[0]

5042

In [146]:
le.classes_[110]

'shankarAbharaNam'

In [147]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np
import os
import librosa
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
def extract_feature(file_name, i):
    X, sample_rate = librosa.load(file_name, offset = 20*i, duration = 10)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft,
    	sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    	sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(indices, ragams, le):
    begin_time = datetime.now()
    features, labels = np.empty((0,193)), np.empty(0) # 193 => total features
    workdir = os.path.join(os.getcwd(), 'songs')
    #le = preprocessing.LabelEncoder()
    #le.fit(ragams)
    j = 0
    begin_time = datetime.now()
    for index, ragam in zip(indices, ragams):
        if j%50==0:
            print(j, datetime.now().strftime("%H:%M:%S"))
        j+=1
        filename = os.path.join(workdir, 'song_{}.mp3'.format(index)) 
        try:
            for i in range(50):
                mfccs, chroma, mel, contrast, tonnetz = extract_feature(filename, i)
                ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
                features = np.vstack([features, ext_features])
                ragam_e = le.transform([ragam])[0]
                labels = np.append(labels, ragam_e)
        except:
            continue
    print(datetime.now()-begin_time)
    return np.array(features, dtype=np.float32), np.array(labels, dtype = np.int8)


df = pd.read_csv('sample_50_rand_df.csv')
X_train_file, X_test_file, y_train_file, y_test_file = train_test_split(df.index, 
                                                    df['Ragam'], 
                                                    test_size = 0.1, 
                                                    random_state = 0,
                                                    stratify = df['Ragam'])
indices = list(X_train_file)
ragams = list(y_train_file)
#print(indices[0], ragams[0], df.loc[indices[0]]['Ragam'])

le = preprocessing.LabelEncoder()
le.fit(ragams)


#features = parse_audio_files(indices[6000:6500], ragams[6000:6500], le)
#np.savetxt('features_13.csv', features[0], delimiter = ',')
#np.savetxt('labels_13.csv', features[1], delimiter = ',')

LabelEncoder()

In [149]:
le.classes_[110]

'shankarAbharaNam'

In [64]:
le = preprocessing.LabelEncoder()
le.fit(ragams)

LabelEncoder()

In [118]:
features = parse_audio_files(indices[:3], ragams[:3])
np.savetxt('features.csv', features[0], delimiter = ',')
np.savetxt('labels.csv', features[1], delimiter = ',')

0 23:11:09
0:02:19.368626


In [109]:
np.savetxt('temp_f.csv', features[0], delimiter = ',')
np.savetxt('temp_r.csv', features[1], delimiter = ',')

In [110]:
temp_f = np.genfromtxt('temp_f.csv', delimiter = ',')
temp_r = np.genfromtxt('temp_r.csv', delimiter = ',')

In [103]:
features[0].shape

(96, 193)

In [104]:
features[1].shape

(96,)

In [101]:
np.savetxt('temp.csv', features, delimiter = ",")

ValueError: could not broadcast input array from shape (96,193) into shape (96)

In [94]:
type(temp_features[1])

numpy.ndarray

In [ ]:
def spectro_gram(file, n_mels = 64, n_fft=1024, hop_len = 1000):
    x, sr = librosa.load(file)
    top_db = 80
    spec = librosa.feature.melspectrogram()

In [ ]:
def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

In [89]:
len(temp_features[0])

64

In [81]:
len(temp_features[1])

126

In [59]:
temp_features[1].shape

(2,)

In [18]:
contrast_1

array([24.06562253, 17.22175933, 20.55255379, 20.87074738, 20.20954483,
       23.67583122, 31.1582898 ])

In [ ]:
chromagram

In [5]:
chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=100, n_chroma=50)

In [6]:
chromagram

array([[0.28605565, 0.28991267, 0.22822933, ..., 0.513678  , 0.60782605,
        0.68441325],
       [0.22866431, 0.25834963, 0.23497471, ..., 0.4396744 , 0.461213  ,
        0.449429  ],
       [0.30673528, 0.33042783, 0.27929902, ..., 0.36918798, 0.42427793,
        0.46696594],
       ...,
       [0.19833562, 0.22123653, 0.18341634, ..., 0.6990352 , 0.7096223 ,
        0.65649253],
       [0.21953504, 0.23454404, 0.19006346, ..., 0.5990146 , 0.62074125,
        0.6032168 ],
       [0.31848478, 0.30835178, 0.22280104, ..., 0.5681043 , 0.6546815 ,
        0.7269734 ]], dtype=float32)

In [3]:
x

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [4]:
sr

22050

In [135]:
temp = np.genfromtxt('features_05.csv', delimiter = ',')

In [136]:
temp.shape

(14465, 193)

In [137]:
temp_l = np.genfromtxt('labels_05.csv', delimiter = ',')

In [142]:
temp_l[:500]

array([ 51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,
        51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,
        51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,
        51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,
        51.,  51.,  51.,  51.,  51., 115., 115., 115., 115., 115., 115.,
       115., 115., 115., 115., 115., 115., 115., 115., 115., 115., 115.,
       115., 115., 115., 115., 115., 115., 115., 115., 115., 115., 115.,
       115., 115., 115., 115., 115., 115., 115., 115., 115., 115., 115.,
       115., 115., 115., 115., 115., 115., 115., 115., 115., 117., 117.,
       117., 117., 117., 117., 117., 117., 117., 117., 117., 117., 117.,
       117., 117., 117., 117., 117., 117., 117., 117., 117., 117.,  75.,
        75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,
        75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,  75.,
        75.,  75.,  75.,  75.,  75.,  75.,  75.,  7

In [134]:
13827 * 5

69135